# Channel Mean + MLP

This is an implementation of Jain et. al. on our dataset

## Reference
Jain, Prakhar, Shubham Bauskar, and Manasi Gyanchandani. "Neural network based non‐invasive method to detect anemia from images of eye conjunctiva." International Journal of Imaging Systems and Technology 30.1 (2020): 112-125.


## Imports

In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas
from tqdm import tqdm

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

## Load Data

In [ ]:
y_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_forniceal_italy.npy") #/30
X_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_forniceal_italy.npy")/255

In [ ]:
y_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_forniceal_india.npy") #/30
X_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_forniceal_india.npy")/255

In [ ]:
## temp
X_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_proc_italy.npy")/255
X_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\X_proc_india.npy")/255

y_it = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_base_italy.npy")
y_in = np.load(r"C:\Users\manas\Documents\Winter 2022\Digital Health Systems\Project\anemia_detection\y_base_india.npy")

## Data Preparation

In [ ]:
X_npy = np.concatenate((X_in, X_it), axis=0)
y_npy = np.concatenate((y_in, y_it), axis=0)/20
X_npy.shape, y_npy.shape

In [ ]:
y_npy[y_npy > 0.75] = 1
y_npy[y_npy < 1] = 0
y_npy

## Channel Mean Computation

In [ ]:
X_r = X_npy[:,:,:,2].mean(axis=(1,2))
X_g = X_npy[:,:,:,1].mean(axis=(1,2))
X_input = np.array([X_r,  X_g]).T

# X_r = X_npy[:,:,:,0].mean(axis=(1,2))
# X_g = X_npy[:,:,:,0].mean(axis=(1,2))
# X_input = np.array([X_r,  X_g]).T

In [ ]:
# X[X==1] = 0
# plt.imshow(X[10])

## ANN/MLP

In [ ]:
from torch.autograd import Variable
# Ref: https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/

In [ ]:
train_split=0.75
val_split = 0.10
test_split = 1 - train_split - val_split
batch_size = 3
epochs = 10

# X=X[:,:,:,0]
# X_in = torch.tensor(X).float().unsqueeze(1).float()
X = X_input
X_input = torch.from_numpy(X).float() #.permute(0,3,2,1)
# X_in.shape()

y_input = torch.tensor(y_npy).float().unsqueeze(1)

print(X_input.shape, y_input.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_use, X_test, y_use,  y_test = train_test_split(X_input, y_input, test_size=test_split)
X_train, X_val, y_train, y_val = train_test_split(X_use, y_use, test_size=val_split/(val_split+train_split))

In [ ]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

## Model Generation

Simple model with 2-6-3-1 structure. Activation sigmoid for 0-1 outputs scaling.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 6)
        self.fc2 = nn.Linear(6, 3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
#         x = F.relu(self.fc2(x))
        return x

## Training

In [ ]:
import torch.optim as optim

net = Net()

criterion = nn.BCELoss()
#nn.L1Loss()
#nn.BCEWithLogitsLoss()
#nn.MSELoss()

optimizer = optim.Adam(net.parameters(), lr=0.01)
#optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#
#
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

## Minor Validation

In [ ]:
y_test.detach().numpy().flatten(), net(X_test).detach().numpy().flatten(), criterion(net(X_test), torch.Tensor(y_test)),

## Full Scale Validation

In [ ]:
batch_size = 10
epochs = 20

# Remove later
net = Net()

net.train()

for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    running_loss = 0.0
    validation_loss = 0.0
    iters = (len(X_train)//batch_size)+1
    
    for i in range((len(X_train)//batch_size)+1):
        # get the inputs; data is a list of [inputs, labels]
        
        try:
            inputs = X_train[(i*batch_size):(i+1)*batch_size]
            labels = y_train[(i*batch_size):(i+1)*batch_size]
        except:
            inputs = X_train[(i*batch_size):]
            labels = y_train[(i*batch_size):]
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
#         print(outputs.shape, labels.shape, outputs.dtype, labels.dtype)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
#     with torch.no_grad():
    y_pred_val = net(Variable(X_val))
    val_loss = criterion(y_pred_val, Variable(torch.Tensor(y_val)))
    validation_loss += val_loss.item()
    
    if epoch %20 == 0 or epoch == epochs-1:
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / iters:.9f}, val_loss: {val_loss.item():.3f}')

print(y_pred_val)
#     running_loss = 0.0
#     validation_loss = 0.0
    #scheduler.step()

In [ ]:
# torch.save(net.state_dict(), 'model_weights.pth')
# model.load_state_dict(torch.load('model_weights.pth'))
# model.eval()

In [ ]:
y_hat = net(X_test).detach().numpy()
y_test = y_test.detach().numpy()
y_hat[y_hat >= 0.75] = 0
y_hat[y_hat > 0] = 1
y_hat, y_test

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(1-y_test, y_hat))

In [ ]:
# y_hat.shape, y_test.shape
criterion(y_hat, y_test)

In [ ]:
nn.L1Loss()(y_hat*30, y_test*30), nn.MSELoss()(y_hat*20, y_test*20)

In [ ]:
net(X_test[5].unsqueeze(0))

In [ ]:
#plt.imshow(X_test[2].permute(2,1,0))

In [ ]:
# for i in range(len(X_test)):
#     print(f'Image: {i+1}, Hb: {y_test[i].item()}, Pred: {y_hat[i].item()}')
#     plt.imshow(X_test[i].permute(2,1,0))
#     plt.figure()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg = LogisticRegression()
reg.fit(X_train, y_train)

In [ ]:
y_val_pred = reg.predict(X_val)
y_val, y_val_pred